In [1]:
from datetime import datetime, timedelta
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from bs4 import BeautifulSoup

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
driver = webdriver.Chrome(chromedriver)

In [3]:
def scrape_weather(begin_date, end_date):
    current_date = datetime.strptime(begin_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')+timedelta(days=1) #add one to make loop end on the end date
    
    lookup_URL = 'https://www.wunderground.com/history/daily/us/il/chicago/KMDW/date/{}-{}-{}'
    
    result = None  #do I need to make result out of main function?
    
    name_of_csv = '{}.csv'.format(begin_date)
    
    while current_date != end_date:
        formatted_lookup_URL = lookup_URL.format(current_date.year, current_date.month, current_date.day)
        #print(formatted_lookup_URL)
        
        driver.get(formatted_lookup_URL)
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'history-observation-table')))
        except TimeoutException:
            print("Loading took too much time! Trying one more time:")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table', {'id': 'history-observation-table'})
        
        df = pd.DataFrame([[td.text.strip() for td in row.findAll('td')] for row in table.tbody.findAll('tr')])
        df['Date'] = current_date
        
        if result is None:
            result = df
        else:
            result = result.append(df).reset_index(drop = True)
        
        result.to_csv(name_of_csv)
        current_date += timedelta(days=1)
    
    return result 

In [4]:
begin_date = '2018-05-08'
end_date = '2019-01-22'

In [ ]:
data_weather = scrape_weather(begin_date, end_date)